# LangChain: Models, Prompts and Output Parsers

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [ ]:
#!pip install python-dotenv
#!pip install openai

In [ ]:
from IPython.display import display, HTML
display(HTML(
"""
<a target="_blank" href="https://colab.research.google.com/github/pedrodiamel/agents-mini-course/blob/course/books/aula_03_model_prompt.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
"""
))

In [38]:
import os
import openai
from utils import format_message, show_prompt


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [6]:
# Set the model variable based on the current date
llm_model = "gpt-4o-mini"

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [7]:
# Ref: https://platform.openai.com/docs/api-reference/chat/create

client = openai.OpenAI()

def get_completion(prompt, model=llm_model):
    messages = [
        {
        "role": "user",
        "content": prompt
        }
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )

    return response.choices[0].message.content


In [8]:
client = openai.OpenAI()

with client.chat.completions.stream(
    model=llm_model,
    messages=[{"role": "user", "content": "Explique brevemente o que é aprendizado de máquina."}],
    temperature=0,
) as stream:
    for event in stream:
        if event.type == "message.delta":
            print(event.delta, end="", flush=True)

In [9]:
response = get_completion("Explique brevemente o que é aprendizado de máquina.")
print(response)

Aprendizado de máquina é um subcampo da inteligência artificial que se concentra no desenvolvimento de algoritmos e modelos que permitem que os computadores aprendam a partir de dados. Em vez de serem programados explicitamente para realizar uma tarefa, os sistemas de aprendizado de máquina identificam padrões e fazem previsões ou decisões com base em exemplos fornecidos. Isso é feito através de técnicas como regressão, classificação e agrupamento, entre outras. O aprendizado de máquina é amplamente utilizado em diversas aplicações, como reconhecimento de voz, recomendação de produtos, detecção de fraudes e muito mais.


In [10]:
styles = ["formal and technical", "casual and friendly", "enthusiastic and persuasive", "concise and to the point", "storytelling and engaging"]
tones = ["confident", "empathetic", "urgent", "optimistic", "serious"]

In [11]:
prompt_task = """
Voce é um assistente de IA que ajuda a redigir emails corporativos. Para o contexto, hoje é {date}.

<Task>
Redigir um email com base no contexto fornecido.
</Task>

O email precisa ter a seguinte estrutura básica:
<Structure>
1. Saudação
2. Introdução ao problema
3. Apresentação da solução
4. Benefícios da solução
5. Chamada para ação (CTA)
6. Despedida
</Structure>
"""

prompt_context = """
<Context>
Escreva um email para um cliente corporativo com o seguinte contexto:
{customer_email}
O email deve ser escrito em um dos seguintes estilos: {style} e tons: {tone}.
</Context>
"""

prompt_instructions = """
<Instrutions>
Use o estilo e tom especificados para redigir o email.
Certifique-se de que o email seja claro, profissional e adequado ao público-alvo.
Não se esqueça de incluir um CTA (Call to Action) para agendar uma demonstração.
Não invente informações; baseie-se apenas no contexto fornecido.
Opcionalmente, use gatilhos mentais com autoridade, urgência, pertencimento, benefício se apropriado.
</Instrutions>
"""

prompt_references = """
<Example>
Por exemplo, um email formal e técnico com tom confiante pode ser:
Asunto: Apresentação de Solução Avançada de Visão Computacional para Inspeção Industrial
Prezado Sr. Silva,
Gostaria de apresentar nossa avançada solução de visão computacional projetada para otimizar processos de inspeção industrial.
Nossa tecnologia utiliza algoritmos de ponta para garantir precisão e eficiência, reduzindo custos operacionais.
Ficaria honrado em agendar uma demonstração para discutir como nossa solução pode beneficiar sua empresa.
Atenciosamente,
João Pereira
</Example>
"""

prompt = f"""{prompt_task}
{prompt_context}
{prompt_instructions}
{prompt_references}
"""


In [12]:
show_prompt(prompt)

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  Voce é um assistente de IA que ajuda a redigir emails corporativos. Para o contexto, hoje é {date}.            │
│                                                                                                                 │
│  <Task>                                                                                                         │
│  Redigir um email com base no contexto fornecido.                                                               │
│  </Task>                                                                                                        │
│                                                                                                                 │
│  O email precisa ter a seguinte estrutura básica:                                                               │
│  <Structure>                                                                                                    │
│  1. Saudação                                                                                                    │
│  2. Introdução ao problema                                                                                      │
│  3. Apresentação da solução                                                                                     │
│  4. Benefícios da solução                                                                                       │
│  5. Chamada para ação (CTA)                                                                                     │
│  6. Despedida                                                                                                   │
│  </Structure>                                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
│  <Context>                                                                                                      │
│  Escreva um email para um cliente corporativo com o seguinte contexto:                                          │
│  {customer_email}                                                                                               │
│  O email deve ser escrito em um dos seguintes estilos: {style} e tons: {tone}.                                  │
│  </Context>                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
│  <Instrutions>                                                                                                  │
│  Use o estilo e tom especificados para redigir o email.                                                         │
│  Certifique-se de que o email seja claro, profissional e adequado ao público-alvo.                              │
│  Não se esqueça de incluir um CTA (Call to Action) para agendar uma demonstração.                               │
│  Não invente informações; baseie-se apenas no contexto fornecido.                                               │
│  Opcionalmente, use gatilhos mentais com autoridade, urgência, pertencimento, benefício se apropriado.          │
│  </Instrutions>                                                                                                 │
│                                                       

In [13]:
customer_email = """
Crie um email para um cliente corporativo apresentando nossa solução
de visão computacional para inspeção industrial. O estilo deve ser
formal e técnico, com tom confiante e persuasivo. Inclua um CTA para
agendar uma demonstração.
"""

print(customer_email)


Crie um email para um cliente corporativo apresentando nossa solução
de visão computacional para inspeção industrial. O estilo deve ser
formal e técnico, com tom confiante e persuasivo. Inclua um CTA para
agendar uma demonstração.



In [14]:
from datetime import datetime
def get_today_str() -> str:
    """Get current date in a human-readable format."""
    return datetime.now().strftime("%a %b %-d, %Y")

prompt.format(customer_email=customer_email, style=styles[0], tone=tones[0], date=get_today_str())
response = get_completion(prompt)

In [15]:
print(response)

Assunto: Proposta de Solução para Otimização de Processos

Prezado [Nome do Cliente],

Espero que este email o encontre bem. 

Identificamos que sua empresa enfrenta desafios na otimização de processos, o que pode impactar a eficiência e a produtividade. Sabemos que, em um mercado competitivo, é crucial encontrar maneiras de melhorar continuamente.

Para abordar essa questão, gostaríamos de apresentar nossa solução inovadora de automação de processos. Esta ferramenta foi desenvolvida para simplificar tarefas repetitivas, permitindo que sua equipe se concentre em atividades de maior valor agregado.

Os benefícios dessa solução incluem a redução de erros operacionais, aumento da produtividade e, consequentemente, uma significativa economia de tempo e recursos. Além disso, nossa equipe de especialistas estará à disposição para garantir uma implementação tranquila e eficaz.

Gostaríamos de convidá-lo para agendar uma demonstração personalizada, onde poderemos mostrar como nossa solução pod

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [17]:
!pip install --upgrade langchain
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.3/496.3 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.8
    Uninstalling langchain-core-1.2.8:
      Successfully uninstalled langchain-core-1.2.8
  Attempting uninstall: langchain
    Found existing installation: langchain 1.2.8
    Uninstalling langchain-1.2.8:
      Successfully uninstalled langchain-1.2.8
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 3.0 MB/s eta 0:00:00


### Model

In [18]:
from langchain_openai import ChatOpenAI

In [19]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(
    temperature=0.0,
    model=llm_model
    )
chat

ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x785933907aa0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x78593390c950>, root_client=<openai.OpenAI object at 0x78594bfefe60>, root_async_client=<openai.AsyncOpenAI object at 0x785933907950>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_organization='org-5qZfsmr4Uz6f4Olo3UqM4Fgb', stream_usage=True)

In [20]:
from langchain_core.messages import HumanMessage

prompt.format(
    customer_email=customer_email,
    style=styles[0],
    tone=tones[0],
    date=get_today_str()
)

response = chat.invoke([HumanMessage(content=prompt)])
print("DONE")

DONE


In [21]:
from utils import format_message, show_prompt

show_prompt(prompt)
format_message([response])

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  Voce é um assistente de IA que ajuda a redigir emails corporativos. Para o contexto, hoje é {date}.            │
│                                                                                                                 │
│  <Task>                                                                                                         │
│  Redigir um email com base no contexto fornecido.                                                               │
│  </Task>                                                                                                        │
│                                                                                                                 │
│  O email precisa ter a seguinte estrutura básica:                                                               │
│  <Structure>                                                                                                    │
│  1. Saudação                                                                                                    │
│  2. Introdução ao problema                                                                                      │
│  3. Apresentação da solução                                                                                     │
│  4. Benefícios da solução                                                                                       │
│  5. Chamada para ação (CTA)                                                                                     │
│  6. Despedida                                                                                                   │
│  </Structure>                                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
│  <Context>                                                                                                      │
│  Escreva um email para um cliente corporativo com o seguinte contexto:                                          │
│  {customer_email}                                                                                               │
│  O email deve ser escrito em um dos seguintes estilos: {style} e tons: {tone}.                                  │
│  </Context>                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
│  <Instrutions>                                                                                                  │
│  Use o estilo e tom especificados para redigir o email.                                                         │
│  Certifique-se de que o email seja claro, profissional e adequado ao público-alvo.                              │
│  Não se esqueça de incluir um CTA (Call to Action) para agendar uma demonstração.                               │
│  Não invente informações; baseie-se apenas no contexto fornecido.                                               │
│  Opcionalmente, use gatilhos mentais com autoridade, urgência, pertencimento, benefício se apropriado.          │
│  </Instrutions>                                                                                                 │
│                                                       

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Assunto: Proposta de Solução para Otimização de Processos                                                       │
│                                                                                                                 │
│ Prezado [Nome do Cliente],                                                                                      │
│                                                                                                                 │
│ Espero que este email o encontre bem.                                                                           │
│                                                                                                                 │
│ Nos últimos meses, temos observado que muitas empresas enfrentam desafios significativos na otimização de seus  │
│ processos internos, o que pode resultar em ineficiências e aumento de custos. Compreendemos que a sua           │
│ organização também pode estar lidando com essas questões.                                                       │
│                                                                                                                 │
│ Para ajudar a resolver esses problemas, gostaríamos de apresentar nossa solução inovadora de automação de       │
│ processos. Esta ferramenta foi desenvolvida para simplificar fluxos de trabalho, reduzir erros e aumentar a     │
│ produtividade da equipe.                                                                                        │
│                                                                                                                 │
│ Os benefícios de implementar nossa solução incluem a redução do tempo gasto em tarefas manuais, a melhoria na   │
│ precisão dos dados e, consequentemente, uma economia significativa nos custos operacionais. Além disso, nossa   │
│ plataforma é escalável e se adapta facilmente às necessidades específicas da sua empresa.                       │
│                                                                                                                 │
│ Gostaríamos de convidá-lo para agendar uma demonstração, onde poderemos mostrar como nossa solução pode ser     │
│ integrada ao seu ambiente de trabalho e trazer resultados tangíveis. Por favor, responda a este email com sua   │
│ disponibilidade, e ficaremos felizes em marcar um horário que seja conveniente para você.                       │
│                                                                                                                 │
│ Agradeço pela atenção e estou à disposição para quaisquer dúvidas.                                              │
│                                                                                                                 │
│ Atenciosamente,                                                                                                 │
│                                                                                                                 │
│ [Seu Nome]                                                                                                      │
│ [Seu Cargo]                                                                                                     │
│ [Nome da Empresa]                                                                                               │
│ [Telefone]                                                                                                      │
│ [Email]                                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Format output

In [28]:
from pydantic import BaseModel, Field

class EmailResponse(BaseModel):
    customer_email: str = Field(..., description="The email content generated for the customer.")
    tips: str = Field(..., description="Additional tips for improving the email.")
    chain_of_thought: str = Field(..., description="The reasoning process behind the email generation.")
    score: int = Field(..., description="A score from 1 to 10 evaluating the quality of the email.")

class EmailImprovedResponse(BaseModel):
    customer_email: str = Field(..., description="The email content generated for the customer.")
    improved_email: str = Field(..., description="The email with the improvements applied")
    chain_of_thought: str = Field(..., description="The reasoning process behind the email generation.")
    score: int = Field(..., description="A score from 1 to 10 evaluating the quality of the email.")

class EmailEvaluationResponse(BaseModel):
    customer_email: str = Field(..., description="The email content generated for the customer.")
    improved_email: str = Field(..., description="The email with the improvements applied")
    chain_of_thought: str = Field(..., description="The reasoning process behind the email generation.")
    score: int = Field(..., description="A score from 1 to 10 evaluating the quality of the applied improvements.")

In [45]:
from langchain.tools import tool
import requests
from langchain_core.messages import (
    HumanMessage,
    ToolMessage,
)

from pydantic import BaseModel, Field

class EmailResponse(BaseModel):
    customer_email: str = Field(..., description="The email content generated for the customer.")
    tips: str = Field(..., description="Additional tips for improving the email.")
    chain_of_thought: str = Field(..., description="The reasoning process behind the email generation.")
    score: int = Field(..., description="A score from 1 to 10 evaluating the quality of the email.")

class EmailImprovedResponse(BaseModel):
    customer_email: str = Field(..., description="The email content generated for the customer.")
    improved_email: str = Field(..., description="The email with the improvements applied")
    chain_of_thought: str = Field(..., description="The reasoning process behind the email generation.")
    score: int = Field(..., description="A score from 1 to 10 evaluating the quality of the email.")

class EmailEvaluationResponse(BaseModel):
    customer_email: str = Field(..., description="The email content generated for the customer.")
    improved_email: str = Field(..., description="The email with the improvements applied")
    chain_of_thought: str = Field(..., description="The reasoning process behind the email generation.")
    score: int = Field(..., description="A score from 1 to 10 evaluating the quality of the applied improvements.")

WEATHER_API_KEY = os.environ['WEATHER_API_KEY']

@tool
def get_weather(city: str, unit: str = "celsius") -> str:
    """
    Obtém a previsão do tempo para a cidade informada.
    Args:
        city: Nome da cidade (ex: 'Recife, PE, Brasil')
        unit: Unidade de temperatura ('fahrenheit', 'celsius', 'kelvin')
    Returns:
        String com resumo da previsão do tempo.
    """

    # Chama API

    # Dummy response
    # resumo = f"[Serviço fictício] A previsão para {city} é: 26°C, céu parcialmente nublado."

    # Real call
    units = {"fahrenheit": "imperial", "celsius": "metric", "kelvin": "standard"}
    if unit not in units:
        raise ValueError("Invalid unit. Must be 'fahrenheit', 'celsius', or 'kelvin'.")

    response = requests.get(f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_API_KEY}&units={units[unit]}")
    data = response.json()
    print(data)
    weather_info = {
        "location": city,
        "temperature": data['main']['temp'],
        "unit": unit,
        "forecast": [data['weather'][0]['description']],
    }

    resumo = f"A temperatura em {city} agora é {weather_info['temperature']}°{unit[0].upper()} com {', '.join(weather_info['forecast'])}."
    return resumo

@tool
def thinking_tool(reflexion:str):
    """Chame a ferramenta de reflexão para analisar a resposta."""
    return f"[Ferramenta de reflexão] Analisando: {reflexion}"

llm = ChatOpenAI(
  model=llm_model,
  temperature=0.7
)


llm_with_tools = llm.bind_tools([thinking_tool, get_weather])
user_input = "Qual a previsão do tempo para Recife hoje em celsius?"

result = llm_with_tools.invoke([HumanMessage(content=user_input)])

# Verifica se houve chamado da tool
if hasattr(result, "tool_calls") and result.tool_calls:

    tool_call = result.tool_calls[0]
    tool_name = tool_call["name"]
    args = tool_call["args"]


    if tool_name != "get_weather":
        raise ValueError(f"Tool inesperada: {tool_name}")

    # Executa a tool
    tool_result = get_weather.invoke(args)

    tool_message = ToolMessage(
        name=tool_name,
        content=str(tool_result),
        tool_call_id=tool_call["id"]
    )


    # Envia de volta ao modelo o resultado da tool, para que ele complete a resposta
    followup = llm_with_tools.invoke([
        HumanMessage(content=user_input),
        result,
        tool_message
    ])

    result = followup.content

else:
    result = result.content

weather = result


email_writer_agent = (
    chat
    .with_structured_output(EmailResponse)
)

email_with_improvements_agent = (
    chat
    .with_structured_output(EmailImprovedResponse)
)

email_evaluator_agent = (
    chat
    .with_structured_output(EmailEvaluationResponse)
)

prompt.format(
    customer_email=customer_email,
    style=styles[0],
    tone=tones[0],
    date=get_today_str()
)

response = email_writer_agent.invoke([HumanMessage(content=prompt)])

improvements_prompt = f"""
Voce é um assistente de IA que ajuda a aprimorar emails corporativos.

<Task>
Aprimorar um email com base nas sugestões sugeridas.
</Task>

<original_email>
{response.customer_email}
</original_email>

<suggestions>
{response.tips}
</suggestions>

<climate_suggestions>
Inclua também essas informações sobre o clima local: {weather}
</climate_suggestions>
"""

improvedResponse = email_with_improvements_agent.invoke([HumanMessage(content=improvements_prompt)])
evaluation_prompt = f"""
Voce é um assistente de IA que avalia melhorias em emails corporativos.

<Task>
Avaliar com nota de 1-10 as melhorias aplicadas.
</Task>

<original_email>
{improvedResponse.customer_email}
</original_email>

<improved_email>
{improvedResponse.improved_email}
</improved_email>
"""

evaluationResponse = email_evaluator_agent.invoke([HumanMessage(content=evaluation_prompt)])
print(f"O email é: {response.customer_email}")
print("\n##########\n")
print(f"As sugestões são: {response.tips}")
print("\n##########\n")
print(f"O email atualizado é: {improvedResponse.improved_email}")
print("\n##########\n")
print(f"A nota final de avaliação, é: {evaluationResponse.score}")


print("DONE")

{'cod': 401, 'message': 'Invalid API key. Please see https://openweathermap.org/faq#error401 for more info.'}


KeyError: 'main'

In [ ]:
response

EmailResponse(customer_email='Assunto: Proposta de Solução para Aumento de Eficiência Operacional\n\nPrezado Sr. Almeida,\n\nEspero que este email o encontre bem. Estou entrando em contato para discutir um desafio que muitas empresas enfrentam atualmente: a necessidade de aumentar a eficiência operacional em um ambiente de negócios cada vez mais competitivo.\n\nPara abordar essa questão, nossa equipe desenvolveu uma solução inovadora que integra tecnologia de automação com análise de dados em tempo real. Essa abordagem não apenas simplifica processos, mas também proporciona insights valiosos que podem ser utilizados para otimizar a tomada de decisões.\n\nOs benefícios dessa solução incluem a redução de custos operacionais, aumento da produtividade e a capacidade de responder rapidamente às mudanças do mercado. Com a implementação dessa tecnologia, sua empresa poderá não apenas se manter competitiva, mas também se destacar em seu setor.\n\nGostaria de convidá-lo para agendar uma demonst

In [31]:
print("Customer Email:")
print(response.customer_email)
print("\nTips:")
print(response.tips)
print("\nChain of Thought:")
print(response.chain_of_thought)
print("\nScore:")
print(response.score)
print("weather")

Customer Email:
Prezado Sr. Almeida,

Espero que esteja bem. Estou entrando em contato para discutir um desafio que muitas empresas enfrentam atualmente: a gestão eficiente de projetos e a colaboração entre equipes.

Para resolver esse problema, nossa plataforma de gerenciamento de projetos oferece uma solução integrada que facilita a comunicação, o acompanhamento de tarefas e a alocação de recursos. Com ferramentas intuitivas e relatórios em tempo real, sua equipe poderá trabalhar de forma mais coesa e produtiva.

Os benefícios dessa solução incluem aumento da eficiência operacional, redução de retrabalho e melhor visibilidade sobre o progresso dos projetos. Além disso, a nossa plataforma é escalável, adaptando-se às necessidades específicas da sua empresa.

Gostaria de convidá-lo para agendar uma demonstração da nossa plataforma, onde poderemos explorar como ela pode transformar a gestão de projetos na sua organização. Por favor, me avise sobre sua disponibilidade.

Agradeço pela ate

### Exercisio

Para aprimorar os resultados obtidos vamos a criar um sistemas multi-agente para conseguir fazer uma análise do problema (reflexão), escrever o email a partir deste plano (escrita) e finalmente avaliar os resultados obtidos (avaliação).

- Step 1: Crie os três agentes (Agente reflexão, Agente escrita, Agente avaliação)
- Step 2: Crie um pipeline (chain) para gerar os resultados
- Step 3: Avalie e intere sobre os resultados obtidos


### Extra: Usando ollama LLMs

In [ ]:
#!pip install langchain-community

In [ ]:
from langchain_community.llms import Ollama

# Conecta ao servidor Ollama (local ou remoto)
llm = Ollama(
    model="gemma2:2b",
    base_url="http://ollama:11434"  # troque se estiver remoto
)

# Prompt simples
response = llm.invoke("Olá! Me diga um 'Olá Mundo' bem bonito em português.")

print("Resposta do modelo:")
print(response)

Resposta do modelo:
Olá, mundo! 🌎✨ 

Que a beleza e o encanto da vida fluam por todo o seu caminho. 😉  😊 

